In [19]:
####
#Clean Porn Dataset
#Tianhao
#2021/7/29
####

import pandas as pd
import datetime
import numpy as np
import xlsxwriter

In [20]:
### Read Categories File
pornhub_categories = pd.read_excel(r'C:\Users\raymo\Documents\GitHub\Callander-Research\Porn database\Data\RAW\PH_gay\Categories.xlsx')

### Read xlsx File
local_path = 'C:/Users/raymo/Documents/GitHub/Callander-Research/Porn database/Data/RAW/PH_gay/' #local path of where the raw data stored
store_path = 'C:/Users/raymo/Documents/GitHub/Callander-Research/Porn database/Data/OUTPUT/' #local path of where the output data stored
file_names = ['2021-03-07.xlsx','2021-03-14.xlsx','2021-03-21.xlsx','2021-03-28.xlsx','2021-04-04.xlsx','2021-04-18.xlsx'
             ,'2021-05-16.xlsx','2021-05-30.xlsx','2021-06-13.xlsx','2021-06-20.xlsx','2021-07-18.xlsx','2021-07-25.xlsx'
             ] #file name

datasets = {}
for file in file_names:
    sample = pd.read_excel(local_path+file,engine=None) #read in files
    datasets[file] = sample



C:\Users\raymo\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [40]:
sample = pd.read_excel('C:/Users/raymo/Documents/GitHub/Callander-Research/Porn database/Data/RAW/PH_gay/2021-06-20.xlsx',engine=None)

C:\Users\raymo\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [180]:
#Series types
#This function convert series into correct types and check any null

def convert_types(file_name, data):
    copy_data = data.copy()

    #extract the viewkeys from pornhub urls
    
    copy_data["url"] = [i.replace("https://www.pornhub.com/view_video.php?viewkey=","") for i in copy_data["url"]]
    
    #convert views(str) to int
    copy_data["views"] = copy_data["views"].fillna('0')
    copy_data["views"] = [int(str(i).replace(',' , '')) for i in copy_data["views"]]
    
    #convert approval to int
    copy_data["approval"] = copy_data["approval"].fillna('0')
    copy_data["approval"] = [int(str(i).replace('%',''))/100 for i in copy_data["approval"]]
    
    
    #convert upload_date to date_time
    extract_time_str = file_name.replace(".xlsx","").split("-")
    extract_time = datetime.date(int(extract_time_str[0]),int(extract_time_str[1]),int(extract_time_str[2]))
    upload_time = np.array([])
    for i in copy_data["upload_date"]:
        copy_data["upload_date"] = copy_data["upload_date"].fillna('0 years ago')
        temp = i.split(" ")
        if temp[0] == 'Yesterday':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= 1))
        elif temp[1] == 'week' or temp[1] == 'weeks':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= int(temp[0])*7))
        elif temp[1] == 'month' or temp[1] == 'months':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= int(temp[0])*30))
        elif temp[1] == 'year' or temp[1] == 'years':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= int(temp[0])*365))
        elif temp[1] == 'day' or temp[1] == 'days':
            upload_time = np.append(upload_time,extract_time- datetime.timedelta(days= int(temp[0])*1))
        else:
            upload_time = np.append(upload_time,'')
    copy_data["upload_date"] = upload_time
    
    #Convert all categories into string and check null
    copy_data["categories"] = [str(i) for i in copy_data["categories"]]
    
    #convert approval_pos and neg to int 
    copy_data["approval_pos"] = copy_data["approval_pos"].fillna('0')
    copy_data["approval_pos"] = [int(str(i).replace('%','').replace('K','000')) for i in copy_data["approval_pos"]]
    copy_data["approval_neg"] = copy_data["approval_neg"].fillna('0')
    copy_data["approval_neg"] = [int(str(i).replace('%','').replace(".0","").replace('K','000')) for i in copy_data["approval_neg"]]
    
    #Calculate True approval rate
    copy_data["approval"] = round(copy_data["approval_pos"]/(copy_data["approval_pos"]+copy_data["approval_neg"]),4)
    
    
    #Clean Actors Column
    copy_data["actors"] = copy_data["actors"].fillna('')
    copy_data["actors"] = [i.replace("Pornstars: ",'').replace("Suggest ",'').replace("\n ",'')
        .replace("Thank you for your suggestions! Our team is reviewing them!","") for i in copy_data["actors"]]
    copy_data["actors"] = ['NaN' if i == '' else i for i in copy_data["actors"]]

    #clean comment number
    copy_data["comments_number"] = copy_data["comments_number"].fillna('0')
    copy_data["comments_number"] = [int(i.replace('(','').replace(')','')) for i in copy_data["comments_number"]]
    
    #drop related_videos
    copy_data = copy_data.drop(columns = ["related_videos"])
    
    #drop videostill_image_alt
    copy_data = copy_data.drop(columns = ["videostill_image_alt"])
    
    #get bag of words for categories
    categories_data = modify_categories(copy_data)
    
    #Clean Comments
    comments = copy_data["comments_text"].fillna('')
    all_comments = []
    for c in comments:
        c_only = []
        temp = c.split(";")
        for i in temp:
            if '\n' not in i:
                c_only.append(i)
        all_comments.append(c_only)
    for i in np.arange(len(comments)):
        if comments[i] == [""]:
            comments[i] = []
    copy_data["comments_text"] = all_comments
    
    #clean related playlists
    
    return copy_data,categories_data


In [164]:
# This function create the bag of words for categories
def modify_categories(data):
    new_categories = [i.replace("Categories \n ","").replace(" \n Suggest","") for i in data["categories"]]
    bag = np.array([[]])
    for fullstring in data["categories"]: 
        temp = np.array([])
        for substring in pornhub_categories["Categories"]:
                
            if substring in fullstring:
                temp = np.append(temp, 1)
            elif substring not in fullstring:
                temp = np.append(temp, 0)
        
        bag = np.append(bag,temp)  
    bag = bag.reshape(data["categories"].size,pornhub_categories["Categories"].size)
    bag = pd.DataFrame(bag,index = data["url"] ,columns = pornhub_categories["Categories"])
    return bag



In [182]:
convert_types("2021-2-3", sample)[0]

,url,title,views,approval,upload_date,approval_pos,approval_neg,uploaded_user,categories,actors,...,comments_number,comments_text,related_playlists,videostill_image,videostill_image_filename,related_videos_videostill,related_videos_videostill_alt,related_playlists_videostill,related_playlists_videostill_alt,_url_input
0,ph606c9b210339e,Fucked Hard Doggy Style at the Window of a Cap...,23033,0.9512,2020-12-05,156,8,Falcon Al,Categories \n Bareback Big Dick Blowjob Gay HD...,NaN,...,2,[ wow. easily one of the top 10 vids of all ti...,XL BB sessions (part 1); AMATEUR; Uncut 3; bar...,https://di.phncdn.com/videos/202104/06/3862300...,videostill_image/(m=eaf8Ggaaaa)(mh=5NqiJJ9XbWs...,https://di.phncdn.com/videos/202104/14/3866059...,"Very eagerly licked the guy's anus, and then h...",https://di.phncdn.com/videos/202106/16/3897291...,XL BB sessions (part 1); AMATEUR; Uncut 3; bar...,https://www.pornhub.com/view_video.php?viewkey...
1,ph5db00da91be01,Fatty Jerking off on the Couch,2659,0.9600,2020-02-04,24,1,Daddy Nox,Categories \n Amateur Bear Chubby Gay HD Porn ...,NaN,...,4,"[ Love your chubby body, love to be in between...",NaN,https://ei.phncdn.com/videos/201910/23/2566842...,videostill_image/(m=eaf8Ggaaaa)(mh=dZSoJ7TP-29...,https://ei.phncdn.com/videos/201908/01/2388875...,2 twinks jerking off together with precum and ...,NaN,NaN,https://www.pornhub.com/view_video.php?viewkey...
2,ph60ace024ad1a7,SOUTHERNSTROKES Christian Hermes and Nick Star...,5159,0.8065,2021-01-13,25,6,maniamen,Categories \n Bareback Big Dick Blowjob Fetish...,NaN,...,0,[],NaN,https://ei.phncdn.com/videos/202105/25/3885710...,videostill_image/(m=eaf8Ggaaaa)(mh=XchgUtHxm99...,https://ei.phncdn.com/videos/202106/08/3892767...,Amateur Twinks Fucking Bareback - Drake and hi...,NaN,NaN,https://www.pornhub.com/view_video.php?viewkey...
3,ph5f7f6e3e7df61,Hammock Blowjob with Cum Shot,6917,0.8298,2020-06-08,39,8,hunguncutfun,Categories \n Amateur Big Dick Blowjob Gay HD ...,NaN,...,2,[],Uncut 3; Oral; SUPERSUCKER 5; cock suckers; ch...,https://di.phncdn.com/videos/202010/08/3590533...,videostill_image/(m=eaf8Ggaaaa)(mh=Lg5iJlASj-m...,https://ci.phncdn.com/videos/202011/25/3734990...,"""Cum Down My Throat"" - Justfor.fans/RafaelMovi...",https://ei.phncdn.com/videos/201404/12/2546893...,Uncut 3; Oral; SUPERSUCKER 5; cock suckers; ch...,https://www.pornhub.com/view_video.php?viewkey...
4,ph5f2ca9248b63e,Up Close with my Uncut Cock & Cum,3175,1.0000,2020-11-05,15,0,Sai Arya,Categories \n Amateur Cumshot Gay HD Porn Hand...,NaN,...,3,"[ great video,thanks]",NaN,https://ei.phncdn.com/videos/202008/07/3401095...,videostill_image/(m=eaf8Ggaaaa)(mh=qlT20wKHznL...,https://ei.phncdn.com/videos/202102/28/3844044...,Lazy Sunday masturbation of a fat curvy guy.; ...,NaN,NaN,https://www.pornhub.com/view_video.php?viewkey...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,ph6086d8a953051,Chavs Fuck at Crusin Spot - Onlyfans/northwest...,88433,0.8913,2021-01-04,443,54,NorthWestChavs,Categories \n Bareback Big Dick Blowjob Creamp...,NaN,...,20,[ That video I enjoyed watching. It gets your ...,Young Muscle; spandex; Raw seeding; Cruising; ...,https://ei.phncdn.com/videos/202104/26/3871599...,videostill_image/(m=eaf8Ggaaaa)(mh=Tb28T4g2TXJ...,https://ei.phncdn.com/videos/201812/31/1993365...,Straight Married Guy Wakes Up to Angel Wolf on...,https://ei.phncdn.com/videos/202102/22/3840811...,Raw seeding; Cruising; Raw; Skinheads; British...,https://www.pornhub.com/view_video.php?viewkey...
993,ph60153fccb8aae,Beautiful Femboy Fucks his own Ass and Cums Big,66316,0.8901,2020-10-06,243,30,prettyboi2000x,Categories \n Amateur Big Dick Gay HD Porn Han...,NaN,...,2,[ Fuck take me from la 7 inch cock ????],Young Muscle; spandex; ggg; gggg; dildo fuckin...,https://ei.phncdn.com/videos/202101/30/3826894...,videostill_image/(m=eaf8Ggaaaa)(mh=MghUQCPfcGF...,https://di.phncdn.com/videos/202008/19/3437890...,Riding A Big Zucchini; Slim teen boy sloppy su..

In [138]:
sample["comments_text"]

nan

In [146]:
x = 0
np.isnan(x)

False

In [25]:
#export modified dataset

for k in datasets.keys():
    modified = convert_types(k,datasets[k])
    
    #write out excel file
    modified[0].to_excel(store_path+"OUTPUT_" + k,sheet_name = 'Sheet 1')
    modified[1].to_excel(store_path+"OUTPUT_Categories_" + k,sheet_name = 'Sheet 2')
    print("Output for file: ", k, " is successful :) ")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Output for file:  2021-03-07.xlsx  is successful :) 


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Output for file:  2021-03-14.xlsx  is successful :) 


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

Output for file:  2021-06-20.xlsx  is successful :) 


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Output for file:  2021-07-18.xlsx  is successful :) 


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Output for file:  2021-07-25.xlsx  is successful :) 
